In [1]:
using CSV
include("../runsetup_tpp.jl");

In [2]:
# Training data
start = 2018, 1, 2
dayslookback = 7
ndays = 365-dayslookback-1
predicttype = :temp
offsets = [1,7,13,19,25];

In [3]:
function run_training(offset)
    X, y = build_multi_data(mdata, predicttype, start, dayslookback, ndays; offset=offset)
    (train_X, train_y), (test_X, test_y) = IAI.split_data(:regression, X, y; seed=123)
    grid = IAI.GridSearch(
        IAI.OptimalTreeRegressor(
            random_seed=123,
            max_depth = 5,
            minbucket = 10,
            show_progress=false
        ),
    show_progress=false  
    )
    IAI.fit!(grid, train_X, train_y)
    lnr = IAI.get_learner(grid)
    IAI.write_json(string("tpp_depth5_",offset,".json"), lnr)
end

run_training (generic function with 1 method)

In [4]:
CSV.write("tpp_depth5_timings.csv", DataFrame(t=[]), delim = ';')
for (idx, offset) in enumerate(offsets)
    t = @elapsed run_training(offset)
    println(idx, " ", t)
    df = DataFrame(t=[t])
    CSV.write("tpp_depth5_timings.csv", df, delim = ';',append=true)
end

┌ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.
└ @ IAILicensing C:\Users\iai\builds\InterpretableAI\SysImgBuilder\.julia\packages\IAILicensing\x1CT6\src\precompile.jl:19
┌ Warning: Interpretable AI license expires soon: 2020-12-31. If you need to renew, please send us the following machine ID:
│ 1d23b3eee464c21ff412aa6a5b247b9f0543ded03e05d39d863fbfc9241e7a1f
└ @ IAILicensing C:\Users\iai\builds\InterpretableAI\SysImgBuilder\.julia\packages\IAILicensing\x1CT6\src\precompile.jl:29


1 1489.7886659
2 943.1893876
3 2541.5330314
4 1520.1659457
5 1645.8399788


In [9]:
function extract_data(name)
    start = 2018, 1, 2
    dayslookback = 7
    ndays = 365-dayslookback-1
    predicttype = :temp
    offsets = [1,7,13,19,25];
    
    df = DataFrame(score_is=[], score_oos=[], score1=[], score2=[]);
    for (idx, offset) in enumerate(offsets)
        X, y = build_multi_data(mdata, predicttype, start, dayslookback, ndays; offset=offset)
        (train_X, train_y), (test_X, test_y) = IAI.split_data(:regression, X, y; seed=123)
        lnr = IAI.read_json(string(name,offset,".json"))
        score = IAI.score(lnr, test_X, test_y) 

        # Testing data
        start1 = 2019, 1, 1
        ndays = 365-dayslookback-1
        test_X1, test_y1 = build_multi_data(mdata, predicttype, start1, dayslookback,
         ndays; offset=offset);

        start2 = 2020, 1, 1
        ndays = 300 # since we don't have days until the end
        test_X2, test_y2 = build_multi_data(mdata, predicttype, start2, dayslookback, 
            ndays; offset=offset)

        score_is = IAI.score(lnr, train_X, train_y)
        score_oos = IAI.score(lnr, test_X, test_y)
        score1 = IAI.score(lnr, test_X1, test_y1)  
        score2 = IAI.score(lnr, test_X2, test_y2)
        push!(df,[score_is, score_oos, score1, score2])
    end
    return df
end

extract_data (generic function with 1 method)

In [12]:
dftpp = extract_data("tpp_depth5_")

,score_is,score_oos,score1,score2
,Any,Any,Any,Any
1,0.989403,0.988495,0.987716,0.985291
2,0.897678,0.893697,0.87149,0.839694
3,0.867718,0.859407,0.809792,0.752408
4,0.854838,0.843922,0.788519,0.75802
5,0.843706,0.841503,0.760264,0.706415


In [19]:
CSV.File("tpp_depth5_timings.csv") |> DataFrame

,t
,Float64
1,1489.79
2,943.189
3,2541.53
4,1520.17
5,1645.84


In [20]:
CSV.File("temp_timings.csv") |> DataFrame

,t
,Float64
1,451.735
2,359.812
3,386.299
4,339.668
5,354.42


In [ ]:
dftemp = extract_data("temp")